###  Libraries

In [ ]:
import os
import sys
import torch
import numpy as np

from Datasets.Datasets_Functions import *
from Visualization import Visualization

from Related_Codes.DECAutoencoder import DECAutoencoder, DECCDAutoencoder
from General_Functions import General_Functions

### Global Variables

In [ ]:
path_to_module = './'
sys.path.append(path_to_module)
os.environ['OMP_NUM_THREADS'] = '6'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

### Load Dataset

In [ ]:
dataset_name = "emnist_balanced_letters_A_J" # The available datasets are: emnist_balanced_digits, emnist_mnist, emnist_balanced_letters_A_J, emnist_balanced_letters_K_T, emnist_balanced_letters_U_Z, har, pendigits, waveform_v1, synthetic
module_name = "Datasets.Datasets"
batch_size = 256
n_clusters = 10
dataloader, input_dim, data_np, labels = function_get_dataset(dataset_name, module_name, batch_size, n_clusters)
print('Data_Shape is:', input_dim)
print('Batches Number is:', len(dataloader))
print(np.unique(labels))

### Visualization

In [ ]:
visualization = Visualization()

### Parameters

In [ ]:
# Set latent dimension and negative slope
latent_dim = 10
negative_slope = 0

# Set pre-training epochs and learning rate
n_pret_epochs = 100
pret_lr = 1e-3

# Set alpha, momentum, training epochs and learning rate
alpha = 1.0
momentum = 0.9
n_epochs = 100
lr = 5e-4

# Set use_pretrain and is_mlp_ae
use_pretrain = True
is_mlp_ae = False

### Create Autoencoder Model

In [ ]:
if(is_mlp_ae):
    dec_autoencoder = DECAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim, negative_slope=negative_slope)
else:  
    input_dim = 1
    dec_autoencoder = DECCDAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim, negative_slope=negative_slope) 
dec_autoencoder.set_general_training_variables(dataloader=dataloader, batch_size=batch_size)
dec_autoencoder.set_pretraining_variables(n_pret_epochs=n_pret_epochs, pret_lr=pret_lr)
dec_autoencoder.set_training_variables(n_epochs=n_epochs, lr=lr, momentum=momentum, alpha=alpha )
dec_autoencoder.set_path_variables(path_to_module=path_to_module, dataset_name=dataset_name)
dec_autoencoder.set_path()
dec_autoencoder = dec_autoencoder.to(device)

### Pretrain Autoencoder

In [ ]:
if(use_pretrain):
    dec_autoencoder.pretrain_autoencoder()
    dec_autoencoder.save_pretrained_weights()
else:
    model_save_path = dec_autoencoder.data_dir_path + '/Weigths/autoencoder_weights.pth'
    dec_autoencoder.load_state_dict(torch.load(model_save_path))

### Initialize the clustering layer using k-means

In [ ]:
dec_autoencoder.kmeans_initialization()

### Train Autoencoder

In [ ]:
dec_autoencoder.train_autoencoder()

### Save Clustering Results

In [ ]:
General_Functions().save_excel(dec_autoencoder.data_dir_path, dec_autoencoder.df_eval)

### Plot Clustering Results

In [ ]:
cluster_centers = dec_autoencoder.get_cluster_centers()
_, dec_autoencoder_reduced_data, labels = dec_autoencoder.get_latent_data()
visualization.plot_tsne(dec_autoencoder_reduced_data, labels, labels, cluster_centers, dec_autoencoder.data_dir_path)